## Critical path showcase



This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

The latter method is recommended as the dependencies found with `from simpy` correspond wth the events as scheduled within the simulation, whereas `from recorded activities` assumes dependencies from recorded data (dependencies assumed in case of identical timestamps and simulation objects).

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (third) vessel may move the amount from site 1 to site 2. The barges do not arrive nor relieve one another at sites at identical times and (hence) the dependencies are identical for both methods.
* four barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (fifth) vessel moves the entire amount from site 1 to site 2. The barges do arrive or relieve each other at sites at identical times - which causes more dependencies with method `from recorded activities` than with method `from simpy`.

Note that in the two barges simulation the barges have an unqiue start event (delay) and velocity - for the four barges simulation this is not the case.

#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, CriticalPathEnvironment
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [4]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 (nr resources=1) to site 2 (nr resources=4) whereafter another vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):            
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading
        
        if nr_barges == 2:
            # alter velocity and start event delay based on index barge
            duration_move = duration - 200*i
            start_time = i*100
        else:
            duration_move = duration
            start_time = 0

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration_move
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":start_time,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 2. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [5]:
# Run the simulation.
simulation_data_out = demo_data(2, 100, env=CriticalPathEnvironment)

In [6]:
# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,77885a04-9668-4a67-bfa5-a27de56bb6ab,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,c92bf677-05a1-41fd-885f-d0f1357aca54,False
9,7190c8b9-997d-4a89-b380-d7861c7ded27,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,33ed2d4c-cb50-4f47-a41d-bd1cdcabc4af,True
15,f71bf49e-7139-4914-9f70-fa15c56862c1,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,dfa447a8-3980-4be1-a1ff-ed01ff5a9f63,False
18,778ea091-bab8-4475-bc01-4314051a1fef,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,f166eeb8-5580-42ff-aba1-1c3639832692,True
21,97d2d279-cfad-46ea-9afc-d1a2a6c10d77,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,f1d8efcd-e9e9-4146-abd4-b34928e1c014,True


In [7]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [8]:
# Run the simulation.
simulation_data_out = demo_data(2, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,5066ba1c-c196-433a-8c34-d3d5978cf3bd,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,ccdd559e-fcac-431e-b03a-2bd0123ba114,False
9,0dcc9743-6486-450f-9052-84c2f56613a2,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,1728c6e4-fef0-4d22-ad3f-128ce499549e,False
15,2325a7dc-300c-4177-b4e5-a78d0a5327fc,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,132cda41-c305-499f-85a7-045791c99b7c,False
18,d0441abd-12b4-4d9c-908e-2a88ae910f17,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,d56bf8a9-e277-44de-b603-07250ae7d15d,False
21,1247da4c-1734-42e2-b6bc-a4a7bedaba7c,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,b458ad52-e5aa-4ddc-8620-2080566c7986,False


In [9]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 4. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 4 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [10]:
# Run the simulation.
simulation_data_out = demo_data(4, 100, env=CriticalPathEnvironment)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,9dc362c4-1b28-4dcd-8e97-b83064a0852d,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,8a7c579c-1814-418b-9480-25aa987250db,True
3,5c874094-c158-4454-9753-97933ba214cc,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,b4923107-5d1f-44fb-9866-7ccaccb188b9,False
5,de0b31c3-5738-4d66-ad07-1ed3522e5cab,basic activity:barge_2,barge_2,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,aeea2b3d-0cce-4f10-9cb1-504ba56a1ec2,False
7,33261483-a9bd-4a1a-805b-f89adea67ff5,basic activity:barge_3,barge_3,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,21dc0c7f-22cc-40c8-b853-359d1c312919,False
26,ae60a90f-0450-44e0-a7f6-58a9d5087be5,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,9134a6ec-0e6a-4e87-9d38-52a141d7e4e6,True


In [11]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 4 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [12]:
# Run the simulation.
simulation_data_out = demo_data(4, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,fa06cea0-092f-401c-8269-7ca3e59f6ca2,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,bff60b87-85f0-4fc7-9a3c-e61c6685f0fd,True
3,9e81debe-70ea-441b-9c33-09e45d2854fc,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,8505a9bc-54fc-4d6d-892f-4c869eb59da2,False
5,d005e50d-d6b0-4b80-b0c0-e196850a9b7c,basic activity:barge_2,barge_2,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,d3733e36-31c5-42a8-8440-36ca549804c6,False
7,e9e657f5-7a75-4c93-abc0-9176936f6685,basic activity:barge_3,barge_3,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,96c4aced-0490-493d-b697-a7c0a1062f92,False
26,bdbd6b49-0215-4563-b6cc-230cab83e675,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,c77fe151-3083-41ab-97a6-f3a449335624,True


In [13]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()